In [ ]:
import copy
import os
import sys
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sklearn

import torch
import torchvision
from torchvision import transforms
import torch.nn as nn

sys.path.append('./../')
from src.dataset import VehiclePredictorDataset
from src.utils import train_model, evaluate_model, get_model

## GPU Check

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

## Config Variables

In [ ]:
root_path = './../'
data_path = os.path.join(root_path, 'data')
dataset_path = os.path.join(data_path, 'VMMRdb')
with open(os.path.join(data_path, 'make_model_most_common_100.pkl'), 'rb') as f:
    target_make_model_labels = pickle.load(f)

# load the dataset for some stats
vp_dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_labels=target_make_model_labels)
num_images = len(vp_dataset)
num_labels = len(vp_dataset.make_model_counts)
class_distribution = vp_dataset.make_model_counts
print(f"num_images: {num_images}")
print(f"num_labels: {num_labels}")

## Define the transforms
- Add other transforms here later, if needed. 
- Do we need any specific transforms for train and val?

In [ ]:
def get_transform():
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return transform

## Instantiate the train, val and test dataloaders

In [ ]:
train_split_ratio = 0.8
val_split_ratio = 0.1
test_split_ration = 0.1

# later see if you can have train-specific transforms
dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_labels=target_make_model_labels, transform=get_transform())

# split dataset in train and val set
dataset_len = len(dataset)
indices = torch.randperm(dataset_len).tolist()
train_split_index = int(train_split_ratio * dataset_len)
val_split_index = train_split_index + int(val_split_ratio * dataset_len)
train_dataset = torch.utils.data.Subset(dataset, indices[0:train_split_index])
val_dataset = torch.utils.data.Subset(dataset, indices[train_split_index:val_split_index])
test_dataset = torch.utils.data.Subset(dataset, indices[val_split_index:])

# define the dataloaders
dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=1,
)

dataloader_val = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=1,
)

dataloader_test = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
)

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(val_dataset)
}

print(f'train: 0 to {train_split_index}\nval: {train_split_index} to {val_split_index}\ntest: {val_split_index} to {dataset_len-1}')
print(f'dataset_sizes : {dataset_sizes}')

## Model config

In [ ]:
# check https://pytorch.org/vision/0.8/models.html to experiment with other backbone model
backbone_model = 'resnet50'

# get the model
model = get_model(num_labels, backbone_model).to(device)

# the reason for computing weights is to account for the class imbalance
weight_distribution = 1 / torch.tensor(list(class_distribution.values()))
weight_distribution = weight_distribution / weight_distribution.sum()
weight_distribution = weight_distribution.to(device)

# define the loss function
criterion = nn.CrossEntropyLoss(weight=weight_distribution)

# define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# define the scheduler
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Train

In [ ]:
train_model(model, dataloader_train, dataloader_val, criterion, optimizer, num_epochs=10)

## Evaluate on test set

In [ ]:
# Load saved model
model_name = 'resnet18_100classes_10epochs.pth'
model = get_model(num_labels, backbone_model).to(device)
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

cm = evaluate_model(model, dataloader_test)

# plot the CM
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm = pd.DataFrame(cm, index=target_make_model_labels, columns=target_make_model_labels)